In [1]:
%load_ext autoreload

In [2]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import iris_utils #TODO remove
import ipywidgets as widgets
from IPython.display import display
from meshcat import Visualizer

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RigidTransform, RollPitchYaw
from pydrake.systems.meshcat_visualizer import ConnectMeshcatVisualizer
from sandbox import rrtiris

In [4]:
from pydrake.all import GenerateSeedingPolytope, GenerateRandomSeedingPolytope
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.multibody.rational_forward_kinematics import FindEpsilonLower, FindEpsilonLowerVector, FindEpsilonUpperVector
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import MosekSolver, MosekSolverDetails, FindEpsTilCollisionOrRedundantForAllIneqs
from time import time

In [5]:
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
vis = Visualizer(zmq_url=zmq_url)
vis.delete()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [6]:

def load_model():
    model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")
    schunk = FindResourceOrThrow("drake/sos_iris_certifier/schunk_wsg_50_block.sdf")
    box_file_1 = FindResourceOrThrow("drake/sos_iris_certifier/assets/shelves.sdf")

    models =[]

    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)

    parser = Parser(plant, scene_graph)
    parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
           "drake/manipulation/models/wsg_50_description/package.xml")))
    models.append(parser.AddModelFromFile(model_file))
    models.append(parser.AddModelFromFile(box_file_1))
    models.append(parser.AddModelFromFile(schunk))

    locs = [ [0,0,0], [0.70, 0, 0.4], [0,0,0]]
    
    idx = 0
    for model in models[:-1]: # WITH S_C_H_U_N_K
#     for model in models:  # WITHOUT S_C_H_U_N_K
        plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(), RigidTransform(locs[idx]))
        idx +=1
    
    #fuse S_C_H_U_N_K
    plant.WeldFrames(plant.get_body(plant.GetBodyIndices(models[0])[-1]).body_frame(),
                    plant.get_body(plant.GetBodyIndices(models[-1])[0]).body_frame(),
                    RigidTransform(RollPitchYaw([np.pi/2,0, np.pi/2]).ToRotationMatrix(),[0,0,0.114]))

    plant.Finalize()
    return plant, builder, scene_graph, models

def load_vis(plant, builder, scene_graph, zmq_url, col_geom = 0):
    vis = Visualizer(zmq_url=zmq_url)
    vis.delete()
    if col_geom == 0:
        viz_role = Role.kIllustration
    else:
        viz_role = Role.kProximity
    visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False, role=viz_role )

    diagram = builder.Build()
    visualizer.load()
    return vis, diagram
plant, builder, scene_graph, models = load_model()
vis, diagram = load_vis(plant, builder, scene_graph, zmq_url, col_geom = 0)
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
sg_context = scene_graph.GetMyContextFromRoot(context)
diagram.Publish(context)


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


In [7]:
#filter_manager = scene_graph.collision_filter_manager(sg_context)
inspector = scene_graph.model_inspector()

#pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
#print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

all_gids = []
for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity):
    #print(gid)
    name = inspector.GetName(gid)
    if "iiwa7::iiwa_link_1" in name:
        print(name)
        link_1 = gid
    if "iiwa7::iiwa_link_0" in name:
        print(name)
        link_0 = gid    
    all_gids.append(gid)
    #gid_name = inspector.GetName(gid)
    
one_zero_var_links = GeometrySet([link_0, link_1])
all_gid_set = GeometrySet(all_gids)
scene_graph.collision_filter_manager().Apply(CollisionFilterDeclaration().ExcludeBetween(one_zero_var_links, all_gid_set))
pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

iiwa7::iiwa_link_1_collision
iiwa7::iiwa_link_0_collision
57 -> 76


In [8]:
import ipywidgets as widgets
from IPython.display import display
q_slider = np.zeros((7,))
sliders = []
for low, up, joint_idx in zip(plant.GetPositionLowerLimits()[:7], plant.GetPositionUpperLimits()[:7], range(7)):
    sliders.append(
        widgets.FloatSlider(
        min=low, 
        max=up, 
        value=0, 
        description=f'iiwa_joint_{joint_idx}'
        )
    )

ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(1e-4, 0.01)
    
def eval_cons(q, c=collision_constraint, tol=0.01):
    return 1 - 1 * float(c.evaluator().CheckSatisfied(q, tol))


def handle_slider_change(change, idx):
    q_slider[idx] = change['new']
    #print(q, end="\r")
#     print(f'In collision: {eval_cons(q_slider)}')
    plant.SetPositions(plant_context, q_slider)
    diagram.Publish(context)
    

for idx, slider in enumerate(sliders):
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
for slider in sliders:
    display(slider)

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [9]:
display(vis.jupyter_cell())

In [10]:
q_low = plant.GetPositionLowerLimits().tolist()
q_high = plant.GetPositionUpperLimits().tolist()
Ratfk = RationalForwardKinematics(plant)

In [11]:
seed_points_q = np.array([
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],  # initial config
    [0.0, 0.5, 0.0, -1.9, 0.0, -0.8, 0.0], # arm in shelf
                       ])
# s = seed_points[1]
# print(eval_cons(s))
# plant.SetPositions(plant_context, s)
# diagram.Publish(context)

In [12]:
seed_points = np.array([Ratfk.ComputeTValue(seed_points_q[idx], np.zeros((3,)))\
                        for idx in range(seed_points_q.shape[0])])


#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), np.zeros((7,)) ))
    
starting_poly = HPolyhedron.MakeBox(limits_t[0], limits_t[1])
context = diagram.CreateDefaultContext()
q_star = np.zeros(7)
def check_empty_polytope(C, d):
    prog = MathematicalProgram()
    t = prog.NewContinuousVariables(C.shape[1], "t")

    prog.AddLinearConstraint(C,-np.inf*np.ones(C.shape[0]), d, t)
    prog.AddBoundingBoxConstraint(limits_t[0], limits_t[1], t)
    result = MosekSolver().Solve(prog)
    return result.is_success(), result.get_solution_result()

## Create default region

In [14]:
regions_default = []
max_vects = []
min_vects = []
vector_bisection_options_list = []
bilinear_alteration_options_list = []
interleaved_options_list = []

#parameters for creating a default region. Currently set to create a box
# num_perm_dim = 0
# num_rot = 1

num_unit_box_copies = 2
initial_box_scale = 0.25
gaussian_variance = 0.1

for i, s in enumerate(seed_points):
#     regions_default.append(GenerateSeedingPolytope(s, num_perm_dim, num_rot))
    regions_default.append(GenerateRandomSeedingPolytope(s,
                                     num_unit_box_copies,
                                     initial_box_scale,
                                     gaussian_variance))
    eps_min_vect = FindEpsilonLowerVector(regions_default[i].A(), 
                                      regions_default[i].b(), 
                                      limits_t[0], limits_t[1], s)
    
    min_vects.append(eps_min_vect)
    
#     eps_max_vect = FindEpsTilCollisionOrRedundantForAllIneqs(plant,
# #                                                              context,
#                                                              plant.GetMyMutableContextFromRoot(context), 
#                                 q_star, regions_default[i].A(), regions_default[i].b(), 
# #                                          eps_min_vect, 
#                                                              limits_t[0], limits_t[1], s)
    eps_max_vect = 10*regions_default[i].b()
    max_vects.append(eps_max_vect)
    
    vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
    vector_bisection_search_options.max_iters = 10
    vector_bisection_search_options.max_feasible_iters = 5
    vector_bisection_search_options.search_d = True
    vector_bisection_search_options.epsilon_min = np.zeros_like(regions_default[i].b())
    vector_bisection_search_options.epsilon_max = eps_max_vect
    vector_bisection_search_options.num_threads = -1
    vector_bisection_options_list.append(vector_bisection_search_options)
    
    bilinear_alternation_options = rational_forward_kinematics.BilinearAlternationOption()
    bilinear_alternation_options.max_iters = 20
    bilinear_alternation_options.lagrangian_backoff_scale = 0
    bilinear_alternation_options.polytope_backoff_scale = 0
    bilinear_alternation_options.num_threads = -1
    bilinear_alteration_options_list.append(bilinear_alternation_options)
    
    binary_search_option = rational_forward_kinematics.BinarySearchOption()
    binary_search_option.epsilon_max = 5
    binary_search_option.epsilon_min = FindEpsilonLower(regions_default[i].A(), 
                                                        regions_default[i].b()+eps_min_vect, 
                                                        limits_t[0], limits_t[1], s)
    binary_search_option.max_iters = 10
    binary_search_option.search_d = True
    binary_search_option.num_threads = 4
    
    interleaved_options = rational_forward_kinematics.InterleavedRegionSearchOptions()
    interleaved_options.scalar_binary_search_options = binary_search_option
    interleaved_options.vector_bisection_search_options = vector_bisection_search_options
    interleaved_options.bilinear_alternation_options = bilinear_alternation_options
    #actually does round robin
    interleaved_options.max_method_switch = 3
    interleaved_options_list.append(interleaved_options)
    
    
num_round_robin_rounds = 1



## Run Cspace Free Region

In [15]:
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)
filtered_collision_pairs = set()
solver_options = mp.SolverOptions()
# solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)

In [ ]:
seed_point_list = [s for s in seed_points] 
print(len(seed_point_list))
C_mats = [r.A() for r in regions_default]
d_vects = [r.b() for r in regions_default]
t0 = time()
cspace_free_region_solution_interleaved_vect = cspace_free_region.InterleavedCSpacePolytopeSearchForSeedPoints(
                                                                    q_star,
                                                                     filtered_collision_pairs,
                                                                     C_mats, d_vects, num_round_robin_rounds, 
                                                                     interleaved_options_list,
                                                                     solver_options, seed_point_list, 
    context = plant.GetMyMutableContextFromRoot(context))
# cspace_free_region_solution_round_robin_vect = cspace_free_region.CspacePolytopeRoundRobinBisectionSearchForSeedPoints(
#                                                                     q_star,
#                                                                      filtered_collision_pairs,
#                                                                      C_mats, d_vects, num_round_robin_rounds, 
#                                                                      vector_bisection_options_list,
#                                                                      solver_options, seed_point_list)
t1 = time()


[2022-02-10 00:02:59.140] [console] [info] starting binary search
[2022-02-10 00:02:59.140] [console] [info] starting binary search


2


[2022-02-10 00:03:06.932] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7::iiwa_link_2_collision, iiwa7::iiwa_link_5_collision)
(iiwa7::iiwa_link_2_collision, iiwa7::iiwa_link_7_collision)
(iiwa7::iiwa_link_2_collision, shelves::right_wall)

[2022-02-10 00:03:07.019] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7::iiwa_link_2_collision, iiwa7::iiwa_link_5_collision)
(iiwa7::iiwa_link_2_collision, iiwa7::iiwa_link_7_collision)
(iiwa7::iiwa_link_2_collision, shelves::right_wall)

[2022-02-10 00:03:52.372] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-10 00:03:52.432] [console] [info] max(power(det(P), 1/8))=2.9446432577914074e-07, solver_time 0.0413668155670166
[2022-02-10 00:03:55.682] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-10 00:03:55.748] [console] [info] max(power(det(P), 1/8))=2.666441303783655e-07, solver_time 0.05015707015991211
[2022-

[2022-02-10 00:11:13.942] [console] [info] epsilon=0.009765624793756242 is feasible
[2022-02-10 00:11:13.943] [console] [info] reset eps_min=0.0, eps_max=0.009765618355479233
[2022-02-10 00:11:21.931] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7::iiwa_link_7_collision, shelves::shelf_lower)
(shelves::shelf_lower, Schunk_Gripper::body_collision)

[2022-02-10 00:11:21.933] [console] [info] epsilon=0.004882811488050864 is infeasible
[2022-02-10 00:11:21.984] [console] [info] max(power(det(P), 1/8))=0.04360830664352522, solver_time 0.03203392028808594
[2022-02-10 00:11:22.215] [console] [info] Starting bisections
[2022-02-10 00:11:42.751] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7::iiwa_link_5_collision, shelves::top)

[2022-02-10 00:11:42.754] [console] [info] epsilon=0.0048828091777396165 is infeasible
[2022-02-10 00:11:42.804] [console] [info] max(power(det(P), 1/8))=0.24023724942124777, solver_time 

In [ ]:
print(seed_points)

In [ ]:
print(f"Completed points in {t1-t0}s")
regions_round_robin = []
ellipses_round_robin = []
for sol in cspace_free_region_solution_interleaved_vect:
    r = HPolyhedron(np.vstack([starting_poly.A(),sol.C]),
                np.hstack([starting_poly.b(), sol.d])
                )
    e = Hyperellipsoid(np.linalg.inv(sol.P), sol.q)

    regions_round_robin.append(r)
    ellipses_round_robin.append(e)

In [ ]:
print(f"Completed points in {t1-t0}s")
regions_round_robin = []
ellipses_round_robin = []
for i, sol in enumerate(cspace_free_region_solution_interleaved_vect):
    dtmp = sol.d+sol.C@seed_points[i,:]
    
    r = HPolyhedron(np.vstack([starting_poly.A(),sol.C*2]),
                np.hstack([starting_poly.b(), dtmp])
                )
    e = Hyperellipsoid(np.linalg.inv(sol.P), sol.q)

    regions_round_robin.append(r)
    ellipses_round_robin.append(e)

In [ ]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("finished running at =", dt_string)	

In [ ]:
iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 20
iris_options.configuration_space_margin = 1e-5
iris_options.max_faces_per_collision_pair = 50
iris_options.termination_threshold = -1
iris_options.relative_termination_threshold = 0.01
iris_options.enable_ibex = False
iris_options.q_star = np.zeros(7)
iris_options.certify_region_with_sos_during_generation = False
iris_options.certify_region_with_sos_after_generation = False
rational_fk = RationalForwardKinematics(plant)
poi = []
for pt in [start, target]:
    poi.append(rational_fk.ComputeTValue(pt, np.zeros((7,)) ))

#compute limits in t-space
limits_t = []
for q in [plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()]:
    limits_t.append(rational_fk.ComputeTValue(np.array(q), np.zeros((7,)) ))
    
starting_poly = HPolyhedron.MakeBox(limits_t[0], limits_t[1])

def iris_handle(seed, domain):
    seed_q = rational_fk.ComputeQValue(np.array(seed), np.zeros((7,)) )
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), seed_q)
    return IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context),
                                              iris_options, domain)

In [ ]:
region = iris_handle(poi[0], starting_poly)
#pull back all faces of poly
b_region_small = region.A()@poi[0] + 1e-5
region_small = HPolyhedron(region.A(), b_region_small)
regions = [region_small]

In [ ]:
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.multibody.rational_forward_kinematics import FindEpsilonLower
from pydrake.solvers import mathematicalprogram as mp
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)

In [ ]:
FindEpsilonLower(limits_t[0], limits_t[1], regions[0].A(), regions[0].b())

In [ ]:
edited_regions = []
filtered_collision_pairs = set()

binary_search_options = rational_forward_kinematics.BinarySearchOption()
binary_search_options.epsilon_max = 10
binary_search_options.epsilon_min = 1e-8
binary_search_options.max_iters = 5
binary_search_options.search_d = True
bilinear_alternation_option = rational_forward_kinematics.BilinearAlternationOption()
bilinear_alternation_option.max_iters = 20
bilinear_alternation_option.lagrangian_backoff_scale = 1e-3
bilinear_alternation_option.polytope_backoff_scale = 1e-5

solver_options = mp.SolverOptions()
solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)
for r in regions: 
    try:
        d_feasible = \
            cspace_free_region.CspacePolytopeBinarySearch(
                iris_options.q_star, filtered_collision_pairs, r.A(), r.b(),
                binary_search_options, solver_options)
        
        #C_final, d_final, P_final, q_final = \
        #    cspace_free_region.CspacePolytopeBilinearAlternation(
        #        iris_options.q_star, filtered_collision_pairs, r.A(), d_feasible,
        #       bilinear_alternation_option, solver_options)
        #C_final = np.vstack([C_final, P_joint_limits.A()])
        #d_final = np.concatenate([d_final, P_joint_limits.b()])
        #editted_regions.append(HPolyhedron(C_final, d_final))
    except Exception as e:
        print(e)
print(len(edited_regions))

In [ ]:
C = regions[0].MaximumVolumeInscribedEllipsoid().A()
radii2, R  =  np.linalg.eig(C.T @ C)
radii = np.sqrt(radii2)

In [ ]:
region.ReduceInequalities


# RRTIRIS SEEDING

In [ ]:
plant.SetPositions(plant_context, start)
diagram.Publish(context)

In [ ]:
ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.001)

def eval_cons(q, c, tol):
    return 1-1*float(c.evaluator().CheckSatisfied(q, tol))
    
col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)

In [ ]:
RRTIRIS = rrtiris.RRTIRIS(poi[0],
                          poi[1],
                          limits = limits_t,
                          default_iris_domain = starting_poly,
                          iris_handle = iris_handle,
                          offset_scaler = 0.8,
                          init_goal_sample_rate = 0.05,
                          goal_sample_rate_scaler = 0.1,
                          verbose = True,
                          plotcallback = None,
                          )
RRTIRIS.run(1)

In [ ]:
regions = RRTIRIS.node_regions

In [ ]:
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.solvers import mathematicalprogram as mp
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)

In [ ]:
editted_regions = []
filtered_collision_pairs = set()

binary_search_options = rational_forward_kinematics.BinarySearchOption()
binary_search_options.epsilon_max = 10
binary_search_options.epsilon_min = -.2
binary_search_options.max_iters = 5
binary_search_options.search_d = True

bilinear_alternation_option = rational_forward_kinematics.BilinearAlternationOption()
bilinear_alternation_option.max_iters = 20
bilinear_alternation_option.lagrangian_backoff_scale = 1e-3
bilinear_alternation_option.polytope_backoff_scale = 1e-5

solver_options = mp.SolverOptions()
for r in regions: 
    try:
        d_feasible = \
            cspace_free_region.CspacePolytopeBinarySearch(
                iris_options.q_star, filtered_collision_pairs, r.A(), r.b(),
                binary_search_options, solver_options)
        C_final, d_final, P_final, q_final = \
            cspace_free_region.CspacePolytopeBilinearAlternation(
                iris_options.q_star, filtered_collision_pairs, r.A(), d_feasible,
                bilinear_alternation_option, solver_options)
        C_final = np.vstack([C_final, P_joint_limits.A()])
        d_final = np.concatenate([d_final, P_joint_limits.b()])
        editted_regions.append(HPolyhedron(C_final, d_final))
    except Exception as e:
        print(e)
print(len(editted_regions))